In [1]:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train,  # .shuffle().select(range(50000)),
        "valid": ds_valid,  # .shuffle().select(range(500))
    }
)

raw_datasets

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1017.54it/s]


Generating train split: 606720 examples [00:09, 65934.76 examples/s]
Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 820.00it/s]
Generating validation split: 3322 examples [00:00, 49997.77 examples/s]


DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 606720
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 3322
    })
})

In [3]:
import torch
import random
from transformers import RobertaTokenizer
from transformers import RobertaForCausalLM, RobertaConfig

random.seed(12345)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

#config = RobertaConfig(max_position_embeddings=1024, hidden_size=768, intermediate_size=768, num_hidden_layers=6)
#roberta_model = RobertaForCausalLM(config)

roberta_model = RobertaForCausalLM.from_pretrained("roberta-base")
roberta_model

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [7]:
roberta_model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.36.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [5]:
context_length = 128

def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3322/3322 [00:22<00:00, 147.28 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 606720
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 3322
    })
})

In [ ]:
from sklearn.metrics import mean_squared_error
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

args = TrainingArguments(
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_steps=10000000,
    eval_steps=10000,
    logging_steps=5000,
    save_total_limit=4,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    weight_decay=0.01,
    output_dir='datasets/function_type_lm_test',
    metric_for_best_model='accuracy')

# Tie the weights of the embedding layer and the decoder layer
# if hasattr(wrapper_model, 'tie_weights'):
#     wrapper_model.tie_weights()

trainer = Trainer(
    roberta_model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    #tokenizer=tokenizer,
)


# train the model
trainer.train() #'datasets/function_type_classifier/checkpoint-15000')